In [17]:
# -*- coding: utf8 -*-
import json
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdknlp_automl.request.v20191111 import RunPreTrainServiceRequest
import sys
import pandas as pd
import json
def aliyun_model(comment, aspect):
    # Initialize AcsClient instance
    client = AcsClient(
      "LTAI5tBDbYayt5mi7qc34YAt",
      "FgmiiRV3D330nWMlTdklXsacd6GyXJ",
      "cn-hangzhou"
    );
    content =  comment
    domain = aspect
    input_params = {
        "input": {
            "content": content,
            "domain": domain,
            "entity": True
        }
    }
    # Initialize a request and set parameters
    request = RunPreTrainServiceRequest.RunPreTrainServiceRequest()
    request.set_ServiceName('ReviewAnalysis')
    request.set_PredictContent(json.dumps(input_params))
    # Print response
    response = client.do_action_with_exception(request)
    resp_obj = json.loads(response)
    predict_result = json.loads(resp_obj['PredictResult'])
    return json.dumps(predict_result['data'])

In [18]:
def get_aliyun(df, aspect):
    res_list = []
    counter = 0
    for index, comment in df.iterrows():
        print(counter)
        #print(comment['comment'])
        try:
            res = aliyun_model(comment['comment'], aspect)
            res_list.append(json.loads(res))
            counter += 1
        except:
            print('Exc')
            print(traceback.print_exc())
            time.sleep(1)
    print(counter)
    return res_list

In [19]:
#第11行加了一句count += 1
def formed(res_list):
    columns = ['father','clause', 'aspect_term', 'opinion_term', 'positive', 'negative', 'category']
    aliyun_polar = pd.DataFrame(columns = columns)
    count = 0
    for res in res_list:
        try:
            aspectItem = res['aspectItem']
        except:
           # print(traceback.print_exc())
            count += 1
            print(res)
            continue
        for clauseItem in aspectItem:
            try:
                clause = clauseItem['clause']
                aspt = []
                opt = []
            except:
                print(traceback.print_exc())
                print(aspectItem)
                continue
            for terms in clauseItem['terms']:
                try:
                    aspectTerm = terms['aspectTerm']
                    opinionTerm = terms['opinionTerm']
                    aspt.append(aspectTerm)
                    opt.append(opinionTerm)
                except:
                    print(traceback.print_exc())
                    continue
            aspterm = ' '.join(aspt)
            opterm = ' '.join(opt)
            positiveProb = clauseItem['positiveProb']
            aspectCategory = clauseItem['aspectCategory']
            negativeProb = clauseItem['negativeProb']
            curdf = [[count,clause,aspterm,opterm,positiveProb,negativeProb,aspectCategory]]
            curdf = pd.DataFrame(curdf, columns = columns)
            aliyun_polar = aliyun_polar.append(curdf)
        count += 1
    return aliyun_polar

In [5]:
data_wan = pd.read_excel('data_wan.xlsx')
data_wan['comment']=data_wan['full_comment']

In [6]:
data_wan

,item_id,comment_id,ts,full_comment,comment
0,612611963718,1136530879455,2021-06-08,妈呀，广告打得响亮的都别买了。一点也不好推开！！！！推开后戳出泥！！！,妈呀，广告打得响亮的都别买了。一点也不好推开！！！！推开后戳出泥！！！
1,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,建议大家不要拍，按压都不动，多大力气都按不动，差评
2,612611963718,1135625069091,2021-06-08,有颗粒状结晶，像沙子一样,有颗粒状结晶，像沙子一样
3,612611963718,1135642433288,2021-06-08,不错，很好！服务也很好。,不错，很好！服务也很好。
4,612611963718,1135621225741,2021-06-08,暂时还没有用，到时候在追评...,暂时还没有用，到时候在追评...
...,...,...,...,...,...
13654,638712575700,1176806355654,2022-05-26,还没用，到货很快 ！都是那个棒子国字看不懂…希望好用，先好评,还没用，到货很快 ！都是那个棒子国字看不懂…希望好用，先好评
13655,638712575700,1175747668678,2022-05-27,收到啦，很不错，非常喜欢，值得购买，(*^__^*) 嘻嘻……,收到啦，很不错，非常喜欢，值得购买，(*^__^*) 嘻嘻……
13656,638712575700,1175835420440,2022-05-28,防晒喷雾是我很喜欢的，包装看着蛮不错，淡淡的香味，上脸成膜速度很快，使用感觉蛮不错 的，值得...,防晒喷雾是我很喜欢的，包装看着蛮不错，淡淡的香味，上脸成膜速度很快，使用感觉蛮不错 的，值得...
13657,638712575700,1176258773263,2022-05-28,这款防晒喷雾效果很好，喷上去很凉快，而且很清爽，真的很好用，喷雾很细腻，喷在脸上很舒服，不刺...,这款防晒喷雾效果很好，喷上去很凉快，而且很清爽，真的很好用，喷雾很细腻，喷在脸上很舒服，不刺...


# 前5k数据

In [1]:
result=get_aliyun(data_wan[:5000], 'beauty')
result_formed=formed(result)

In [24]:
#保存数据
result_formed.to_csv("前5000条数据结果.csv")

In [35]:
result_formed

,father,clause,aspect_term,opinion_term,positive,negative,category,item_id,comment_id,ts,full_comment,clasue,aspect,polarity,opinionTerm
0,0,广告打得响亮的都别买了,,别买了,0.000,1.000,其他,612611963718,1136530879455,2021-06-08,妈呀，广告打得响亮的都别买了。一点也不好推开！！！！推开后戳出泥！！！,广告打得响亮的都别买了,,negative,别买了
0,0,一点也不好推开,推开后,不好推开 戳出泥,0.000,1.000,使用效果,612611963718,1136530879455,2021-06-08,妈呀，广告打得响亮的都别买了。一点也不好推开！！！！推开后戳出泥！！！,一点也不好推开,推开后,negative,不好推开 戳出泥
0,1,建议大家不要拍,,不要拍,0.000,1.000,其他,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,建议大家不要拍,,negative,不要拍
0,1,差评,,差评,0.000,1.000,整体,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,差评,,negative,差评
0,1,按压都不动,多大力气 多大力气 按压,都 按不动 不动,0.001,0.999,使用效果,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,按压都不动,多大力气 多大力气 按压,negative,都 按不动 不动
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4999,持久度挺好不脱妆,持久度 持久度,挺好 很不错 不脱妆,0.997,0.003,妆感,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,持久度挺好不脱妆,持久度 持久度,positive,挺好 很不错 不脱妆
0,4999,⌬̢২推荐购买,,推荐购买,1.000,0.000,其他,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,⌬̢২推荐购买,,positive,推荐购买
0,4999,真的变白了⌬̢২珠光很明显🎁,珠光 质地,很明显 奶油一样,0.988,0.010,质地,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,真的变白了⌬̢২珠光很明显🎁,珠光 质地,positive,很明显 奶油一样
0,4999,用后效果明显⌬̢২适合黑黄皮,黑黄皮,适合,0.997,0.002,适合,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,用后效果明显⌬̢২适合黑黄皮,黑黄皮,positive,适合


In [34]:
#根据data_wan填充
result_formed['item_id']=''
result_formed['comment_id']=''
result_formed['ts']=''
result_formed['full_comment']=''
result_formed['clasue']=result_formed['clause']
result_formed['aspect']=result_formed['aspect_term']
result_formed['polarity']=''
result_formed['opinionTerm']=result_formed['opinion_term']
item_id=[]
comment_id=[]
ts=[]
full_comment=[]
polarity=[]
for i in range(len(result_formed['item_id'])):
    full_comment.append(data_wan['full_comment'][list(result_formed['father'])[i]])
    item_id.append(data_wan['item_id'][list(result_formed['father'])[i]])
    comment_id.append(data_wan['comment_id'][list(result_formed['father'])[i]])
    ts.append(data_wan['ts'][list(result_formed['father'])[i]])
    if (float(list(result_formed['negative'])[i])>float(list(result_formed['positive'])[i])):
        polarity.append('negative')
    else:
        polarity.append('positive')

result_formed['item_id']=item_id
result_formed['comment_id']=comment_id
result_formed['ts']=ts
result_formed['full_comment']=full_comment
result_formed['polarity']=polarity


In [36]:
result_end=result_formed[['item_id','comment_id','ts','full_comment','clasue','aspect','polarity','opinionTerm']]

In [44]:
result_end.to_csv("前5000条处理后的数据结果.csv",encoding = 'utf_8_sig')

# 5001条到1w条数据 

In [72]:
#result_2=get_aliyun(data_wan[5000:10001], 'beauty')
result_formed_2=formed(result_2)

{}
{}


In [73]:
len(result_formed_2)

10359

In [23]:
#保存数据
result_formed_2.to_csv("5000-10000条数据结果.csv")
#result_formed_2= pd.read_csv("5000-10000条数据结果.csv")

In [9]:
result_formed_2['father']=result_formed_2['father']+5000

In [10]:
result_formed_2

,Unnamed: 0,father,clause,aspect_term,opinion_term,positive,negative,category
0,0,5001,没有破损,包装,挺好 没有破损,1.000,0.000,包装
1,0,5001,还是挺防水的,NaN,挺防水,0.999,0.001,使用效果
2,0,5001,味道非常好闻,味道 味道,喜欢 非常好闻,1.000,0.000,异味/气味
3,0,5001,喷在手上有美白的效果,喷在手上,有美白的效果,1.000,0.000,美白效果
4,0,5002,物流挺快的,物流,挺快,1.000,0.000,物流
...,...,...,...,...,...,...,...,...
10354,0,9999,喜欢,NaN,喜欢,1.000,0.000,整体
10355,0,9999,物流很快,物流,很快,1.000,0.000,物流
10356,0,10000,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,NaN,没有美白效果,0.002,0.998,美白效果
10357,0,10000,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,喷头,像个高压水枪,0.000,1.000,其他


In [11]:
data_wan[5000:10001]

,item_id,comment_id,ts,full_comment,comment
5000,587991649073,1169310209428,2022-03-20,冲代言人去的,冲代言人去的
5001,587991649073,1170121955040,2022-03-20,包装挺好的，没有破损，味道非常好闻，是我喜欢的味道，喷在手上有美白的效果，还是挺防水的。当然...,包装挺好的，没有破损，味道非常好闻，是我喜欢的味道，喷在手上有美白的效果，还是挺防水的。当然...
5002,587991649073,1170158791930,2022-03-20,物流挺快的，防晒也挺好用,物流挺快的，防晒也挺好用
5003,587991649073,1170161463608,2022-03-20,喷雾很细腻，商品气味真的好香啊，不黏、好推开、味道正好！要夸一下小高，看得见摸得着的效果，一...,喷雾很细腻，商品气味真的好香啊，不黏、好推开、味道正好！要夸一下小高，看得见摸得着的效果，一...
5004,587991649073,1170118571465,2022-03-20,姐妹们冲冲冲 客服“芸芸”简直太喜欢了,姐妹们冲冲冲 客服“芸芸”简直太喜欢了
...,...,...,...,...,...
9996,591484378514,1169540125518,2022-03-22,挺好的，价格适中，质量看着不错,挺好的，价格适中，质量看着不错
9997,591484378514,1169543589709,2022-03-22,很好用，去年也是买这个,很好用，去年也是买这个
9998,591484378514,1169597733963,2022-03-23,持久效果不错，粉色的味道很好闻，喜欢玫瑰的味道,持久效果不错，粉色的味道很好闻，喜欢玫瑰的味道
9999,591484378514,1170457419955,2022-03-23,一直在用这个牌子的防晒，物流很快，也是正品，喜欢。,一直在用这个牌子的防晒，物流很快，也是正品，喜欢。


In [12]:
#根据data_wan填充
result_formed_2['item_id']=''
result_formed_2['comment_id']=''
result_formed_2['ts']=''
result_formed_2['full_comment']=''
result_formed_2['clasue']=result_formed_2['clause']
result_formed_2['aspect']=result_formed_2['aspect_term']
result_formed_2['polarity']=''
result_formed_2['opinionTerm']=result_formed_2['opinion_term']
item_id=[]
comment_id=[]
ts=[]
full_comment=[]
polarity=[]
for i in range(len(result_formed_2['item_id'])):
    full_comment.append(data_wan['full_comment'][list(result_formed_2['father'])[i]])
    item_id.append(data_wan['item_id'][list(result_formed_2['father'])[i]])
    comment_id.append(data_wan['comment_id'][list(result_formed_2['father'])[i]])
    ts.append(data_wan['ts'][list(result_formed_2['father'])[i]])
    if (float(list(result_formed_2['negative'])[i])>float(list(result_formed_2['positive'])[i])):
        polarity.append('negative')
    else:
        polarity.append('positive')

result_formed_2['item_id']=item_id
result_formed_2['comment_id']=comment_id
result_formed_2['ts']=ts
result_formed_2['full_comment']=full_comment
result_formed_2['polarity']=polarity


In [13]:
result_end_2=result_formed_2[['item_id','comment_id','ts','full_comment','clasue','aspect','polarity','opinionTerm']]

In [14]:
result_end_2

,item_id,comment_id,ts,full_comment,clasue,aspect,polarity,opinionTerm
0,587991649073,1170121955040,2022-03-20,包装挺好的，没有破损，味道非常好闻，是我喜欢的味道，喷在手上有美白的效果，还是挺防水的。当然...,没有破损,包装,positive,挺好 没有破损
1,587991649073,1170121955040,2022-03-20,包装挺好的，没有破损，味道非常好闻，是我喜欢的味道，喷在手上有美白的效果，还是挺防水的。当然...,还是挺防水的,NaN,positive,挺防水
2,587991649073,1170121955040,2022-03-20,包装挺好的，没有破损，味道非常好闻，是我喜欢的味道，喷在手上有美白的效果，还是挺防水的。当然...,味道非常好闻,味道 味道,positive,喜欢 非常好闻
3,587991649073,1170121955040,2022-03-20,包装挺好的，没有破损，味道非常好闻，是我喜欢的味道，喷在手上有美白的效果，还是挺防水的。当然...,喷在手上有美白的效果,喷在手上,positive,有美白的效果
4,587991649073,1170158791930,2022-03-20,物流挺快的，防晒也挺好用,物流挺快的,物流,positive,挺快
...,...,...,...,...,...,...,...,...
10354,591484378514,1170457419955,2022-03-23,一直在用这个牌子的防晒，物流很快，也是正品，喜欢。,喜欢,NaN,positive,喜欢
10355,591484378514,1170457419955,2022-03-23,一直在用这个牌子的防晒，物流很快，也是正品，喜欢。,物流很快,物流,positive,很快
10356,591484378514,1170490139615,2022-03-23,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,NaN,negative,没有美白效果
10357,591484378514,1170490139615,2022-03-23,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,喷头,negative,像个高压水枪


In [15]:
result_end_2.to_csv("前5000-10000条处理后的数据结果.csv",encoding = 'utf_8_sig')

# 最后的数据

In [2]:
result_3=get_aliyun(data_wan[10000:15000], 'beauty')
result_formed_3=formed(result_3)

In [24]:
#保存数据
result_formed_3.to_csv("10000条之后的数据结果.csv")
#result_formed_2= pd.read_csv("5000-10000条数据结果.csv")

In [26]:
result_formed_3['father']=result_formed_3['father']+10000

In [27]:
result_formed_3

,father,clause,aspect_term,opinion_term,positive,negative,category
0,10000,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,,没有美白效果,0.002,0.998,美白效果
0,10000,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,喷头,像个高压水枪,0.000,1.000,其他
0,10000,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,呲的 使用感,脖子疼 非常不好,0.000,1.000,使用效果
0,10005,透明清爽不油腻,,透明,0.999,0.001,颜色
0,10005,透明清爽不油腻,,清爽,1.000,0.000,使用效果
...,...,...,...,...,...,...,...
0,13657,很满意,,很满意,1.000,0.000,整体
0,13657,喷雾很细腻,喷雾,很细腻,1.000,0.000,质量
0,13658,,,相信天猫旗舰店，已查验，是正品,0.998,0.002,正品
0,13658,包装很好,包装,很好,1.000,0.000,包装


In [28]:
data_wan[10000:100001]

,item_id,comment_id,ts,full_comment,comment
10000,591484378514,1170490139615,2022-03-23,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...,打开后只喷了脖子 没有美白效果 防晒效果未知 感觉这个喷头像个高压水枪 呲的我脖子疼 使...
10001,591484378514,1169178752915,2022-03-23,还没用，先点了,还没用，先点了
10002,591484378514,1170096846410,2022-03-23,不知是不是正品,不知是不是正品
10003,591484378514,1169174120971,2022-03-23,持久情况：，xx,持久情况：，xx
10004,591484378514,1170487923476,2022-03-23,n次回购,n次回购
...,...,...,...,...,...
13654,638712575700,1176806355654,2022-05-26,还没用，到货很快 ！都是那个棒子国字看不懂…希望好用，先好评,还没用，到货很快 ！都是那个棒子国字看不懂…希望好用，先好评
13655,638712575700,1175747668678,2022-05-27,收到啦，很不错，非常喜欢，值得购买，(*^__^*) 嘻嘻……,收到啦，很不错，非常喜欢，值得购买，(*^__^*) 嘻嘻……
13656,638712575700,1175835420440,2022-05-28,防晒喷雾是我很喜欢的，包装看着蛮不错，淡淡的香味，上脸成膜速度很快，使用感觉蛮不错 的，值得...,防晒喷雾是我很喜欢的，包装看着蛮不错，淡淡的香味，上脸成膜速度很快，使用感觉蛮不错 的，值得...
13657,638712575700,1176258773263,2022-05-28,这款防晒喷雾效果很好，喷上去很凉快，而且很清爽，真的很好用，喷雾很细腻，喷在脸上很舒服，不刺...,这款防晒喷雾效果很好，喷上去很凉快，而且很清爽，真的很好用，喷雾很细腻，喷在脸上很舒服，不刺...


In [29]:
#根据data_wan填充
result_formed_3['item_id']=''
result_formed_3['comment_id']=''
result_formed_3['ts']=''
result_formed_3['full_comment']=''
result_formed_3['clasue']=result_formed_3['clause']
result_formed_3['aspect']=result_formed_3['aspect_term']
result_formed_3['polarity']=''
result_formed_3['opinionTerm']=result_formed_3['opinion_term']
item_id=[]
comment_id=[]
ts=[]
full_comment=[]
polarity=[]
for i in range(len(result_formed_3['item_id'])):
    full_comment.append(data_wan['full_comment'][list(result_formed_3['father'])[i]])
    item_id.append(data_wan['item_id'][list(result_formed_3['father'])[i]])
    comment_id.append(data_wan['comment_id'][list(result_formed_3['father'])[i]])
    ts.append(data_wan['ts'][list(result_formed_3['father'])[i]])
    if (float(list(result_formed_3['negative'])[i])>float(list(result_formed_3['positive'])[i])):
        polarity.append('negative')
    else:
        polarity.append('positive')

result_formed_3['item_id']=item_id
result_formed_3['comment_id']=comment_id
result_formed_3['ts']=ts
result_formed_3['full_comment']=full_comment
result_formed_3['polarity']=polarity


In [31]:
result_end_3=result_formed_3[['item_id','comment_id','ts','full_comment','clasue','aspect','polarity','opinionTerm']]

In [32]:
result_end_3.to_csv("10000条之后处理后的数据结果.csv",encoding = 'utf_8_sig')

In [36]:
a = pd.read_csv('前5000条处理后的数据结果.csv')
b = pd.read_csv('前5000-10000条处理后的数据结果.csv')
c = pd.read_csv('10000条之后处理后的数据结果.csv')


In [41]:
a

,Unnamed: 0,item_id,comment_id,ts,full_comment,clasue,aspect,polarity,opinionTerm
0,0,612611963718,1136530879455,2021-06-08,妈呀，广告打得响亮的都别买了。一点也不好推开！！！！推开后戳出泥！！！,广告打得响亮的都别买了,NaN,negative,别买了
1,0,612611963718,1136530879455,2021-06-08,妈呀，广告打得响亮的都别买了。一点也不好推开！！！！推开后戳出泥！！！,一点也不好推开,推开后,negative,不好推开 戳出泥
2,0,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,建议大家不要拍,NaN,negative,不要拍
3,0,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,差评,NaN,negative,差评
4,0,612611963718,1135620093697,2021-06-08,建议大家不要拍，按压都不动，多大力气都按不动，差评,按压都不动,多大力气 多大力气 按压,negative,都 按不动 不动
...,...,...,...,...,...,...,...,...,...
11169,0,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,持久度挺好不脱妆,持久度 持久度,positive,挺好 很不错 不脱妆
11170,0,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,⌬̢২推荐购买,NaN,positive,推荐购买
11171,0,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,真的变白了⌬̢২珠光很明显🎁,珠光 质地,positive,很明显 奶油一样
11172,0,587991649073,1170127931294,2022-03-20,防晒效果挺好⌬̢২可以个是正品，很百搭，真的变白了⌬̢২珠光很明显🎁，香香的，用后效果明显⌬...,用后效果明显⌬̢২适合黑黄皮,黑黄皮,positive,适合


In [45]:
#保存数据
pd.concat([a,b,c],axis=0).reset_index(drop=True, inplace=True).to_csv("所有结果.csv",encoding = 'utf_8_sig')
#result_formed_2= pd.read_csv("5000-10000条数据结果.csv")

AttributeError: 'NoneType' object has no attribute 'to_csv'